<img src="https://ksassets.timeincuk.net/wp/uploads/sites/54/2018/02/Pubg-920x518.jpg">

# Introduction
PlayerUnknown's BattleGrounds (PUBG), an online multiplayer battle royale game has enjoyed massive popularity. With over 50 million copies sold, it's the fifth best selling game of all time and has millions of active monthly players.

The project aims to predict if a player would make it to the top 10 position in the game. Based on the player's after game statistics, the model would be able to predict the player position. The results could then be published on an online portal and in media prior to any tournaments happening for cash prizes.

What's the best strategy to be in the top ten positions in PUBG? Should you sit in one spot and hide your way into victory, or do you need to be the top shot? Let's let the data do the talking!


# Dataset

The team at PUBG has made official game data available for the public to explore and scavenge outside of "The Blue Circle." The data was pulled from Kaggle which itself collected the data through the official PUBG Developer API - https://developer.pubg.com/apps?locale=en

The dataset has column winPlacePerc, from which we are inferring our target variable that is_top_ten ( Player qualifies to  be in top 10)


## Data fields

* DBNOs - Number of enemy players knocked.
* assists - Number of enemy players this player damaged that were killed by teammates.
* boosts - Number of boost items used.
* damageDealt - Total damage dealt. Note: Self inflicted damage is subtracted.
* headshotKills - Number of enemy players killed with headshots.
* heals - Number of healing items used.
* Id - Player’s Id
* killPlace - Ranking in match of number of enemy players killed.
* killPoints - Kills-based external ranking of player. (Think of this as an Elo ranking where only kills matter.) If there is a value other than -1 in rankPoints, then any 0 in killPoints should be treated as a “None”.
* killStreaks - Max number of enemy players killed in a short amount of time.
* kills - Number of enemy players killed.
* longestKill - Longest distance between player and player killed at time of death. This may be misleading, as downing a player and driving away may lead to a large longestKill stat.
* matchDuration - Duration of match in seconds.
* matchId - ID to identify match. There are no matches that are in both the training and testing set.
* matchType - String identifying the game mode that the data comes from. The standard modes are “solo”, “duo”, “squad”, “solo-fpp”, “duo-fpp”, and “squad-fpp”; other modes are from events or custom matches.
* rankPoints - Elo-like ranking of player. This ranking is inconsistent and is being deprecated in the API’s next version, so use with caution. Value of -1 takes place of “None”.
* revives - Number of times this player revived teammates.
* rideDistance - Total distance traveled in vehicles measured in meters.
* roadKills - Number of kills while in a vehicle.
* swimDistance - Total distance traveled by swimming measured in meters.
* teamKills - Number of times this player killed a teammate.
* vehicleDestroys - Number of vehicles destroyed.
* walkDistance - Total distance traveled on foot measured in meters.
* weaponsAcquired - Number of weapons picked up.
* winPoints - Win-based external ranking of player. (Think of this as an Elo ranking where only winning matters.) If there is a value other than -1 in rankPoints, then any 0 in winPoints should be treated as a “None”.
* groupId - ID to identify a group within a match. If the same group of players plays in different matches, they will have a different groupId each time.
* numGroups - Number of groups we have data for in the match.
* maxPlace - Worst placement we have data for in the match. This may not match with numGroups, as sometimes the data skips over placements.
* winPlacePerc - This is a percentile winning placement, where 1 corresponds to 1st place, and 0 corresponds to last place in the match. It is calculated off of maxPlace, not numGroups, so it is possible to have missing chunks in a match.
* is_top_ten - The target of prediction. Is the player present in top 10 position.


## Dataset Initial Review

In [1]:
# Installing packages not installed by Azure Notebooks by default.
# The execution of this cell could take some time.
# While the cell executes, an asterisk (*) is displayed within the square brackets to the left.

install.packages("https://rattle.togaware.com/src/contrib/rattle_5.2.0.tar.gz")
install.packages("FSelector")
install.packages("rpart.plot")

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
inferring 'repos = NULL' from 'pkgs'
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [2]:
# Load required packages from local library into the R session.

library(dplyr) # Data wrangling, glimpse() and tbl_df().
library(ggplot2) # Visualise data.
library(lubridate) # Dates and time.
library(randomForest) # Impute missing values with na.roughfix().
library(readr) # Efficient reading of CSV data.
library(rattle) # normVarNames().
library(magrittr) # Pipes %>%, %T>% and equals(), extract().
library(scales) # Format comma().
library(ggplot2) # Visualise data.
library(tidyr) # Prepare a tidy dataset, gather().
library(stringi) # String concat operator %s+%.
library(FSelector) # Feature selection: information.gain().
library(tibble) # Convert row names into a column.
library(lubridate) # Dates and time.
library(GGally) # extension to ggplot2
library(gridExtra) # grid evaluation
library(rpart) #rpart function
library(ROCR) #roc curve



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date

randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:ggplot2’:

    margin

The following object is masked from ‘package:dplyr’:

    combine

Rattle: A free graphical interface for data science with R.
Version 5.2.0 Copyright (c) 2006-2018 Togaware Pty Ltd.
Type 'rattle()' to shake, rattle, and roll your data.

Attaching package: ‘rattle’

The following object is masked from ‘package:randomForest’:

    importance


Attaching package: ‘scales’

The following object is masked from ‘package:readr’:

    col_factor


Attaching package: ‘tidyr’

The following object is masked from ‘package:magrittr’:


In [3]:
# Data Ingestion

system.time(playersData <- read.csv("pubgData.csv"))

dsName <- "playersData"
ds <- get(dsName)



   user  system elapsed 
 89.918   1.100  94.262 

In [4]:
names(ds) #variables present in the dataset

# Normalizing Variable Names

names(ds) %<>% normVarNames()

# Confirm the results are as expected.

names(ds)

# Data Review

glimpse(ds)

[1] "X"               "Id"              "groupId"         "matchId"        
 [5] "assists"         "boosts"          "damageDealt"     "DBNOs"          
 [9] "headshotKills"   "heals"           "killPlace"       "killPoints"     
[13] "kills"           "killStreaks"     "longestKill"     "matchDuration"  
[17] "matchType"       "maxPlace"        "numGroups"       "rankPoints"     
[21] "revives"         "rideDistance"    "roadKills"       "swimDistance"   
[25] "teamKills"       "vehicleDestroys" "walkDistance"    "weaponsAcquired"
[29] "winPoints"       "winPlacePerc"

[1] "x"                "id"               "group_id"         "match_id"        
 [5] "assists"          "boosts"           "damage_dealt"     "dbnos"           
 [9] "headshot_kills"   "heals"            "kill_place"       "kill_points"     
[13] "kills"            "kill_streaks"     "longest_kill"     "match_duration"  
[17] "match_type"       "max_place"        "num_groups"       "rank_points"     
[21] "revives"          "ride_distance"    "road_kills"       "swim_distance"   
[25] "team_kills"       "vehicle_destroys" "walk_distance"    "weapons_acquired"
[29] "win_points"       "win_place_perc"

Observations: 200,000
Variables: 30
$ x                <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,...
$ id               <fct> 7f96b2f878858a, eef90569b9d03c, 1eaf90ac73de72, 46...
$ group_id         <fct> 4d4b580de459be, 684d5656442f9e, 6a4a42c3245a74, a9...
$ match_id         <fct> a10357fd1a4a91, aeb375fc57110c, 110163d8bb94ae, f1...
$ assists          <int> 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
$ boosts           <int> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 2,...
$ damage_dealt     <dbl> 0.000, 91.470, 68.000, 32.900, 100.000, 100.000, 0...
$ dbnos            <int> 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,...
$ headshot_kills   <int> 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...
$ heals            <int> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 2, 0, 14...
$ kill_place       <int> 60, 57, 47, 75, 45, 44, 96, 48, 64, 74, 75, 64, 37...
$ kill_points      <int> 1241, 0, 0, 0, 0, 0, 1262, 1000, 0, 0, 0, 0, 0, 98...
$ kills         

In [5]:
# The target variable is_top_ten which we will create based win_place_perc

ds <- ds %>% 
  mutate(is_top_ten = if_else(ds$win_place_perc >= 0.9, TRUE, FALSE))

glimpse(ds) #taking a glimpse after a new variable has been added.

# assigning is_top_ten as target variable
target <- ds$is_top_ten

#visualising and checking the distribution of target varibale in the dataset
ds %>%
  ggplot(aes_string(x=target)) +
  geom_bar(width=0.2, fill="grey") +
  theme(text=element_text(size=14)) +
  scale_y_continuous(labels=comma) +
  labs(title = "Distribution of is_top_ten",
       x = "is_top_ten",
       y = "Count",
       caption = "Source: PUBG DATASET")
nrow(subset(ds, target == TRUE))

Observations: 200,000
Variables: 31
$ x                <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,...
$ id               <fct> 7f96b2f878858a, eef90569b9d03c, 1eaf90ac73de72, 46...
$ group_id         <fct> 4d4b580de459be, 684d5656442f9e, 6a4a42c3245a74, a9...
$ match_id         <fct> a10357fd1a4a91, aeb375fc57110c, 110163d8bb94ae, f1...
$ assists          <int> 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
$ boosts           <int> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 2,...
$ damage_dealt     <dbl> 0.000, 91.470, 68.000, 32.900, 100.000, 100.000, 0...
$ dbnos            <int> 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,...
$ headshot_kills   <int> 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...
$ heals            <int> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 2, 0, 14...
$ kill_place       <int> 60, 57, 47, 75, 45, 44, 96, 48, 64, 74, 75, 64, 37...
$ kill_points      <int> 1241, 0, 0, 0, 0, 0, 1262, 1000, 0, 0, 0, 0, 0, 98...
$ kills         

[1] 21055

In [6]:
# Exploring The Shape of the Data

dim(ds) #rows and columns

nrow(ds) %>% comcat() #number of rows
ncol(ds) %>% comcat() #number of columns
head(ds) #first 6 observations
tail(ds) #last 6 observations

[1] 200000     31

200,000 
31 


x,id,group_id,match_id,assists,boosts,damage_dealt,dbnos,headshot_kills,heals,...,ride_distance,road_kills,swim_distance,team_kills,vehicle_destroys,walk_distance,weapons_acquired,win_points,win_place_perc,is_top_ten
1,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,...,0.0000,0,0.00,0,0,244.80,1,1466,0.4444,FALSE
2,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,...,0.0045,0,11.04,0,0,1434.00,5,0,0.6400,FALSE
3,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,...,0.0000,0,0.00,0,0,161.80,2,0,0.7755,FALSE
4,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,...,0.0000,0,0.00,0,0,202.70,3,0,0.1667,FALSE
5,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,...,0.0000,0,0.00,0,0,49.75,2,0,0.1875,FALSE
6,ff79c12f326506,289a6836a88d27,bac52627a12114,0,0,100.00,1,1,0,...,0.0000,0,0.00,0,0,34.70,1,0,0.0370,FALSE


,x,id,group_id,match_id,assists,boosts,damage_dealt,dbnos,headshot_kills,heals,...,ride_distance,road_kills,swim_distance,team_kills,vehicle_destroys,walk_distance,weapons_acquired,win_points,win_place_perc,is_top_ten
199995,199995,4606207e47bb2e,c942948e2462b7,5d67777e68da16,1,0,200.00,1,0,0,...,0,0,0,0,0,221.40,2,0,0.3333,FALSE
199996,199996,8a03aca8bca167,71ff27619423a3,40c91483da5bb9,0,0,0.00,0,0,0,...,0,0,0,0,0,1531.00,7,0,0.4000,FALSE
199997,199997,982176c1463624,d334f56b3a33be,cfc5059075d4ff,0,0,34.36,0,0,0,...,0,0,0,0,0,11.68,1,0,0.0778,FALSE
199998,199998,7f55e229133735,b4c4099f227e4d,de2f69ee694142,0,2,327.90,0,1,0,...,2482,0,0,0,0,3065.00,8,0,0.9891,TRUE
199999,199999,bb6519ba90106c,a8712a3317b739,455a0249beebf8,0,2,311.50,3,1,1,...,3636,0,0,0,0,1187.00,5,0,0.7407,FALSE
200000,200000,72741410249ab1,d01b0edd11028c,3f4df182b877ff,2,5,631.80,5,0,5,...,0,0,0,0,0,1761.00,6,1509,0.8571,FALSE


In [7]:
# assigning the target variable "is_top_ten" to target

target <- ds$is_top_ten

# find all the types of variables present in the dataset

sapply(ds, class)

x               id         group_id         match_id 
       "integer"         "factor"         "factor"         "factor" 
         assists           boosts     damage_dealt            dbnos 
       "integer"        "integer"        "numeric"        "integer" 
  headshot_kills            heals       kill_place      kill_points 
       "integer"        "integer"        "integer"        "integer" 
           kills     kill_streaks     longest_kill   match_duration 
       "integer"        "integer"        "numeric"        "integer" 
      match_type        max_place       num_groups      rank_points 
        "factor"        "integer"        "integer"        "integer" 
         revives    ride_distance       road_kills    swim_distance 
       "integer"        "numeric"        "integer"        "numeric" 
      team_kills vehicle_destroys    walk_distance weapons_acquired 
       "integer"        "integer"        "numeric"        "integer" 
      win_points   win_place_perc       is_top_ten 
       "integer"        "numeric"        "logical"

## Variable Analysis

In [8]:
# Anaylse variables of dataset

"All Variable"
names(ds)

"Kills Summary"
summary(ds$kills)

"Walk Distance Summary"
summary(ds$walk_distance)

"Weapons Acquired Summary"
summary(ds$weapons_acquired)

"Match Duration Summary"
summary(ds$match_duration)

"Match Type Summary"
table(ds$match_type)

[1] "All Variable"

[1] "x"                "id"               "group_id"         "match_id"        
 [5] "assists"          "boosts"           "damage_dealt"     "dbnos"           
 [9] "headshot_kills"   "heals"            "kill_place"       "kill_points"     
[13] "kills"            "kill_streaks"     "longest_kill"     "match_duration"  
[17] "match_type"       "max_place"        "num_groups"       "rank_points"     
[21] "revives"          "ride_distance"    "road_kills"       "swim_distance"   
[25] "team_kills"       "vehicle_destroys" "walk_distance"    "weapons_acquired"
[29] "win_points"       "win_place_perc"   "is_top_ten"

[1] "Kills Summary"

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0000  0.0000  0.0000  0.9252  1.0000 48.0000 

[1] "Walk Distance Summary"

...

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  0.000   2.000   3.000   3.672   5.000  72.000 

[1] "Match Duration Summary"

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    152    1367    1438    1579    1851    2237 

[1] "Match Type Summary"


        crashfpp         crashtpp              duo          duo-fpp 
             276               16            13846            44901 
        flarefpp         flaretpp       normal-duo   normal-duo-fpp 
              37              109               10              235 
     normal-solo  normal-solo-fpp     normal-squad normal-squad-fpp 
              13               84               26              771 
            solo         solo-fpp            squad        squad-fpp 
            8222            24259            28158            79037 

## Dataset Observations

* The data has 1,00,000 rows and 31 columns. 
* The data has variables which is a mixture of integer, numeric variable types.
* There are some variables like ids, groupids, x which doesn't give any information about data modelling so we may want to remove those data points.

* Most number of kills by a player in a game is 48 and minimum is 0.
* Player walked a maximum distance of 13530 metres in the game
* A player aquires maximum of 72 weapons in a game match
* Match duration ranges from ~2.5 minutes to 37 minutes.
* Most preferred match type is "squad_fpp" and least preferred match type is "normal_solo"


# Data Wrangling

Data wrangling, sometimes referred to as data munging, is the process of transforming and mapping data from one "raw" data form into another format with the intent of making it more appropriate and valuable for a variety of downstream purposes such as analytics.

There are variables which take a particular set of values and can be converted to factor variables.


## Data cleaning


In [9]:
# How many variables in the dataset have unique values and can be coverted to factors

sapply(ds, function(x) length(unique(x)))

# choose assists,  match_type, vehicle_destroys, road_kills and kill_streaks for factor variable conversion

vnames <- c("assists","dbnos", "headshot_kills", "boosts", "revives", "match_type", "vehicle_destroys", "road_kills", "kill_streaks", "is_top_ten")

x               id         group_id         match_id 
          200000           200000           190420            46976 
         assists           boosts     damage_dealt            dbnos 
              14               18            17413               26 
  headshot_kills            heals       kill_place      kill_points 
              16               41              100             1157 
           kills     kill_streaks     longest_kill   match_duration 
              32               11            19793             1206 
      match_type        max_place       num_groups      rank_points 
              16               99               99             1193 
         revives    ride_distance       road_kills    swim_distance 
              15            15794                8             8861 
      team_kills vehicle_destroys    walk_distance weapons_acquired 
               6                5            24880               58 
      win_points   win_place_perc       is_top_ten 
             706             1876                2

In [54]:
# Convert to factor variables

ds[vnames] %<>%
  lapply(factor)

# Note which variables are categoric.

ds %>%
  sapply(is.factor) %>%
  which() %T>%
  print() ->
  catc

# Normalise the levels of all categoric variables.

for (v in catc)
  levels(ds[[v]]) %<>% normVarNames()

# Review the categoric variables to confirm normalisation of levels.

# Ensure target is a factor
# Note the target variable.

target <- "is_top_ten"

# Ensure the target is categoric.

ds[[target]] %<>% as.factor()

# Confirm the distribution.

ds[target] %>% table()

              id         group_id         match_id          assists 
               2                3                4                5 
          boosts            dbnos   headshot_kills     kill_streaks 
               6                8                9               14 
      match_type          revives       road_kills vehicle_destroys 
              17               21               23               26 
      is_top_ten 
              31 


.
 false   true 
178945  21055 

In [11]:
# Check if there is NA in the target variable 
sapply(ds, is.na) %>% sum()
# 1 such value, remove that value from ds

ds <- subset(ds, !is.na(ds$is_top_ten)) 

# Found no such record

[1] 0

## Feature Selection

We identify the  variables (features or columns of the dataset) that are irrelevant or inappropriate for modelling.



In [12]:
# Note the available variables and assign to the variable vars.

ds %>%
  names() %T>%
  print() ->
  vars

# Place the target variable is_top_ten at the beginning of the vars.

c(target, vars) %>%
  unique() %T>%
  print() ->
  vars

# collect the identifiers.

id <- c("x", "id", "group_id", "match_id", "match_type", "win_place_perc")

id %>%
  union(if (exists("risk")) risk) %T>%
  print() ->
  ignore

 [1] "x"                "id"               "group_id"         "match_id"        
 [5] "assists"          "boosts"           "damage_dealt"     "dbnos"           
 [9] "headshot_kills"   "heals"            "kill_place"       "kill_points"     
[13] "kills"            "kill_streaks"     "longest_kill"     "match_duration"  
[17] "match_type"       "max_place"        "num_groups"       "rank_points"     
[21] "revives"          "ride_distance"    "road_kills"       "swim_distance"   
[25] "team_kills"       "vehicle_destroys" "walk_distance"    "weapons_acquired"
[29] "win_points"       "win_place_perc"   "is_top_ten"      
 [1] "is_top_ten"       "x"                "id"               "group_id"        
 [5] "match_id"         "assists"          "boosts"           "damage_dealt"    
 [9] "dbnos"            "headshot_kills"   "heals"            "kill_place"      
[13] "kill_points"      "kills"            "kill_streaks"     "longest_kill"    
[17] "match_duration"   "match_type"       "max

In [13]:
# Defining a helper function to count the number of distinct values in a column of a dataset

count_unique <- function(x)
{
  x %>% unique() %>% length()
}

# Using the heuristic in the above cell, identify the variables which have all unique values and assign their names to ids.

ds[vars] %>%
  sapply(count_unique) %>%
  equals(nrow(ds)) %>%
  which() %>%
  names() %T>%
  print() ->
  ids

"variables to ignore"
ignore <- union(ignore, ids) %T>% print()

[1] "x"  "id"


[1] "variables to ignore"

[1] "x"              "id"             "group_id"       "match_id"      
[5] "match_type"     "win_place_perc"


In [14]:
# Helper function to count the number of values missing.

count_na <- function(x)
{
  x %>% is.na() %>% sum()
}

# Heuristic to identify variables with only missing values.

ds[vars] %>%
  sapply(count_na) %>%
  equals(nrow(ds)) %>%
  which() %>%
  names() %T>%
  print() ->
  missing

# Add them to the variables to be ignored for modelling.

ignore %<>% union(missing) %T>% print()

character(0)
[1] "x"              "id"             "group_id"       "match_id"      
[5] "match_type"     "win_place_perc"


In [15]:
# Identify a threshold above which proportion missing is fatal.

missing.threshold <- 0.5

# Identify variables that are mostly missing.

ds[vars] %>%
  sapply(count_na) %>%
  '>'(missing.threshold*nrow(ds)) %>%  # This line checks if count_na(column) > threshold*nrow(ds)
  which() %>%
  names() %T>%
  print() ->
  mostly

# Add them to the variables to be ignored for modelling.

ignore <- union(ignore, mostly) %T>% print()

character(0)
[1] "x"              "id"             "group_id"       "match_id"      
[5] "match_type"     "win_place_perc"


In [16]:
# Helper function to count the number of levels.
# extract2(x) extracts the column with the name stored in x, from the dataset

count_levels <- function(x)
{
  ds %>% extract2(x) %>% levels() %>% length()
}


# Identify a threshold above which we have too many levels.

levels.threshold <- 20

# Identify variables that have too many levels.

ds[vars] %>%
  sapply(is.factor) %>%            # Returns TRUE for factor variables
  which() %>%
  names() %>%                      # Returns the names of all the factor variables 
  sapply(count_levels)%>%
  '>='(levels.threshold) %>%
  which() %>%
  names() %T>%
  print() ->
  too.many

ignore <- union(ignore, too.many) %T>% print()

[1] "id"       "group_id" "match_id" "dbnos"   
[1] "x"              "id"             "group_id"       "match_id"      
[5] "match_type"     "win_place_perc" "dbnos"         


In [17]:
#Helper function to test if all values in vector are the same.

all_same <- function(x)
{
  all(x == x[1L])   # Checks if all the values in the column equals the value at x[1]
}

# Identify variables that have a single value.

ds[vars] %>%
  sapply(all_same) %>%
  which() %>%
  names() %T>%
  print() ->
  constants

# Add them to the variables to be ignored for modelling.

ignore <- union(ignore, constants) %T>% print()

character(0)
[1] "x"              "id"             "group_id"       "match_id"      
[5] "match_type"     "win_place_perc" "dbnos"         


In [18]:
# Note which variables are numeric and store to the variable numc

vars %>%
  setdiff(ignore) %>%
  magrittr::extract(ds, .) %>%
  sapply(is.numeric) %>%
  which() %>%
  names() %T>%
  print() ->
  numc

# For numeric variables generate a table of correlations

ds[numc] %>%
  cor(use="complete.obs") %>%
  ifelse(upper.tri(., diag=TRUE), NA, .) %>%
  abs() %>%
  data.frame() %>%
  tbl_df() %>%
  set_colnames(numc) %>%
  mutate(var1=numc) %>%
  gather(var2, cor, -var1) %>%
  na.omit() %>%
  arrange(-abs(cor)) %T>%
  print() ->
  mc

 [1] "damage_dealt"     "heals"            "kill_place"       "kill_points"     
 [5] "kills"            "longest_kill"     "match_duration"   "max_place"       
 [9] "num_groups"       "rank_points"      "ride_distance"    "swim_distance"   
[13] "team_kills"       "walk_distance"    "weapons_acquired" "win_points"      
# A tibble: 120 x 3
   var1          var2           cor
   <chr>         <chr>        <dbl>
 1 num_groups    max_place    0.998
 2 win_points    rank_points  0.994
 3 win_points    kill_points  0.983
 4 rank_points   kill_points  0.975
 5 kills         damage_dealt 0.891
 6 kills         kill_place   0.729
 7 kill_place    damage_dealt 0.676
 8 longest_kill  kills        0.604
 9 walk_distance kill_place   0.591
10 longest_kill  damage_dealt 0.564
# ... with 110 more rows


* We identify pairs of variables where we want to keep one but not the other variable because they are highly correlated.
We limit the removals to those correlations that are 0.90 or more.
* The varibales "num_groups" , "win_points", "rank_points" would be added to the ignore list of varibale.

In [19]:
correlated <- c("num_groups", "win_points", "rank_points")

ignore <- union(ignore, correlated) %T>% print()

 [1] "x"              "id"             "group_id"       "match_id"      
 [5] "match_type"     "win_place_perc" "dbnos"          "num_groups"    
 [9] "win_points"     "rank_points"   


### Remove the varibales

In [20]:
# Check the number of variables currently.

length(vars)

# Remove the variables to ignore.

vars %<>% setdiff(ignore) %T>% print()

# Confirm they are now ignored.

length(vars)

[1] 31

 [1] "is_top_ten"       "assists"          "boosts"           "damage_dealt"    
 [5] "headshot_kills"   "heals"            "kill_place"       "kill_points"     
 [9] "kills"            "kill_streaks"     "longest_kill"     "match_duration"  
[13] "max_place"        "revives"          "ride_distance"    "road_kills"      
[17] "swim_distance"    "team_kills"       "vehicle_destroys" "walk_distance"   
[21] "weapons_acquired"


[1] 21

In [21]:
# Construct the formulation of the modelling to undertake.

form <- formula(target %s+% " ~ .") %T>% print()

# Use correlation search to identify key variables.
# This might take some time to execute.

cfs(form, ds[vars])

# Use information gain to identify variable importance.

information.gain(form, ds[vars]) %>%
  rownames_to_column("variable") %>%
  arrange(-attr_importance)

is_top_ten ~ .


[1] "assists"        "boosts"         "headshot_kills" "heals"         
[5] "kill_place"     "longest_kill"   "walk_distance"

variable,attr_importance
walk_distance,0.1136367717
kill_place,0.1108970343
boosts,0.0944355890
damage_dealt,0.0585635049
longest_kill,0.0560147431
kills,0.0538940127
heals,0.0461789644
weapons_acquired,0.0436367740
kill_streaks,0.0349176093
assists,0.0294178655


Looking at the above importance chart, we find that walk_distance, kill_place and boosts have the most importance.

# Data Exploration

Use visualisations to explore the data. Intersperse code/output with your notes and observations.

In [51]:
# Setting default plot size for this notebook.
# Use this line of code to set the size of your visualisation plots.

options(repr.plot.width = 10, repr.plot.height = 4)

In [47]:
names(ds)

[1] "x"                "id"               "group_id"         "match_id"        
 [5] "assists"          "boosts"           "damage_dealt"     "dbnos"           
 [9] "headshot_kills"   "heals"            "kill_place"       "kill_points"     
[13] "kills"            "kill_streaks"     "longest_kill"     "match_duration"  
[17] "match_type"       "max_place"        "num_groups"       "rank_points"     
[21] "revives"          "ride_distance"    "road_kills"       "swim_distance"   
[25] "team_kills"       "vehicle_destroys" "walk_distance"    "weapons_acquired"
[29] "win_points"       "win_place_perc"   "is_top_ten"

In [52]:
# Plot the kills by the player distribution 
ds %>%
    ggplot(aes(x=kills)) +
    geom_bar()


# Plot the kills vs the is_top_ten by the player 
ds %>%
    ggplot(aes(x=kills, fill=is_top_ten)) +
    geom_bar()



Observations
* Most of the player die without killing anybody in the game.
* Players with kills in the range of 6-8 have higher probability of coming in top ten.


In [58]:
options(repr.plot.width = 20, repr.plot.height = 8)

ds %>%
    ggplot(aes(x=match_type, y=kills, fill=match_type, srt=45)) +
    geom_bar(stat="summary", fun.y="mean") +
    theme(legend.position="none")

From the above graph we infer, a player kills more number of players while playing solo.

In [26]:
solos <- subset(ds, num_groups > 50) %>% length()
duos <- subset(ds, num_groups > 25 && num_groups <= 50) %>% length()
squads <- subset(ds, num_groups <= 25) %>% length()

"There are " %s+% solos %s+% " solos"
"There are " %s+% duos %s+% " duos"
"There are " %s+% squads %s+% " squads"

[1] "There are 31 solos"

[1] "There are 31 duos"

[1] "There are 31 squads"

In [57]:
options(repr.plot.width = 20, repr.plot.height = 8)

ds %>%
    ggplot(aes(x=match_type, y=kills, fill=target)) +
    geom_violin() +
    theme(legend.position="none")

In [1]:
ds %>%
    ggplot(aes(x=boosts, y=win_place_perc, fill=is_top_ten)) +
    geom_boxplot(notch=FALSE) +
    theme(legend.position="none") +
    labs(title = "Boxplot for boosts",x = "boosts")


ERROR: Error in ds %>% ggplot(aes(x = boosts, y = win_place_perc)): could not find function "%>%"


To survive as a winner must actively leverage other items or use boost items to increase their power.

# Predictive Modeling

We will partition the full dataset into three: train (70%), validate (15%), test (15%).


In [29]:
# Record the number of observations.

nobs <- nrow(ds) %T>% comcat()

# Create a trainin dataset of 70% of the observation

nobs %>%
  sample(0.70*nobs) %T>% 
  {length(.) %>% comma() %>% cat("Size of training dataset:", ., "\n")} ->
  train

# Create a validation dataset of 15% of the observations.

nobs %>%
  seq_len() %>% 
  setdiff(train) %>% 
  sample(0.15*nobs) %T>%
  {length(.) %>% comma() %>% cat("Size of validation dataset:", ., "\n")} ->
  validate

# Create a testing dataset of 15% (the remainder) of the observations.

nobs %>%
  seq_len() %>%
  setdiff(union(train, validate)) %T>%
  {length(.) %>% comma() %>% cat("Size of testing dataset:", ., "\n")} ->
  test

200,000 
Size of training dataset: 140,000 
Size of validation dataset: 30,000 
Size of testing dataset: 30,000 


In [30]:
# Cache the various actual values for target

tr_target <- ds[train,][[target]]    %T>% {head(., 20) %>% print()}

va_target <- ds[validate,][[target]] %T>% {head(., 20) %>% print()}

te_target <- ds[test,][[target]]     %T>% {head(., 20) %>% print()}

 [1] false false false false false true  false false false false false true 
[13] false false false false false false false false
Levels: false true
 [1] false false false false false false false false false false false false
[13] false false false false false false false false
Levels: false true
 [1] false false false false true  true  false false false false false false
[13] false false false false false false false false
Levels: false true


In [31]:
# Splitting function: "anova" "poisson" "class" "exp"

mthd <- "class"

# Splitting function parameters.

prms <- list(split="information")

# Control the training.

ctrl <- rpart.control(maxdepth=5)

# Build the model

m_rp <- rpart(form, ds[train, vars], method=mthd, parms=prms, control=ctrl)

In [32]:
# Capture the model in generic variables.

model <- m_rp
mtype <- "rpart"
mdesc <- "Decision Tree"

# Visually expose the discovered knowledge.
options(repr.plot.width = 8, repr.plot.height = 6)

fancyRpartPlot(model)

In [33]:
# Review which importance of the variables.

ggVarImp(model)

Walk distance has the greatest impact in deciding if the player would be in top 10.

## Evaluation


In [34]:
# create a function predict probabily and class of varibales
predict_prob_class <- function(data_type)
{
  model %>%
    predict(newdata=ds[data_type, vars], type="prob") %>%
    .[,2] %>%
    set_names(NULL) %>%
    round(2) ->
    prob
  
  model %>%
    predict(newdata=ds[data_type, vars], type="class") %>%
    set_names(NULL) ->
    class
  
  predict_list <- list("probability" = prob, "class" = class)
  return(predict_list)
  
}

In [35]:
# Function to evaluate performance, in terms of accuracy , precison, recall, fscore etc
perf <- function(mdesc, dstype, target, prob, class, verbose=TRUE)
{
  # Provide informative introduction.
  
  if (verbose)
    "Performance Evaluation\n" %s+%
    "======================\n\n" %s+%
    "Model: " %s+% mdesc %s+% "\n" %s+%
    "Dataset: " %s+% dstype %s+% " dataset with " %s+%
    comma(length(prob)) %s+% " observations.\n" %>%
    cat("\n")
  
  # Calculate accuracy and error rates.
  
  sum(class == target, na.rm=TRUE) %>%
    divide_by(class %>% is.na() %>% not() %>% sum()) ->
    acc
  
  sum(class != target, na.rm=TRUE) %>%
    divide_by(class %>% is.na() %>% not() %>% sum()) ->
    err
  
  if (verbose)
    "Overall accuracy: " %s+% percent(acc) %s+% "\n" %s+%
    "Overall error: " %s+% percent(err) %s+% "\n" %>%
    cat("\n")
  
  # Generate error matricies.
  matrix <- errorMatrix(target, class)
  
  if (verbose)
  {
    cat("Error Matrices:\n\n")
    errorMatrix(target, class, count=TRUE) %>% print()
    cat("\n")
    matrix %>% print()
    cat("\n")
  }
  
  # Calculate recall, precision and F-score.
  
  rec <- (matrix[2,2]/(matrix[2,2] + matrix[2,1]))
  pre <- (matrix[2,2]/(matrix[2,2] + matrix[1,2]))
  fsc <- ((2 * pre * rec)/(rec + pre))
  
  if (verbose)
    "Recall: " %s+% percent(rec) %s+% "\n" %s+%
    "Precision: " %s+% percent(pre) %s+% "\n" %s+%
    "F-Score: " %s+% round(fsc, 3) %s+% "\n" %>%
    cat("\n")
  
  # Calculate AUC for the ROC curve.
  
  prob %>%
    prediction(target) %>%
    performance("auc") %>%
    attr("y.values") %>%
    .[[1]] ->
    auc
  
  if (verbose)
    "Percentage area under the ROC curve AUC: " %s+%
    percent(auc) %>%
    cat("\n")
  
  prob %>%
    prediction(target) %>%
    performance("tpr", "fpr") ->
    rates
  
  # Return a list of the evaluations.
  invisible(list(acc=acc,
                 err=err,
                 matrix=matrix,
                 rec=rec,
                 pre=pre,
                 auc=auc,
                 rates=rates))
  
}

In [36]:
# function to create auc plot

aucplot <- function(rates, auc, mdesc, dstype)
{
  data.frame(tpr=attr(rates, "y.values")[[1]],
             fpr=attr(rates, "x.values")[[1]]) %>%
    ggplot(aes(fpr, tpr)) +
    geom_line() +
    labs(title="ROC - " %s+% mtype %s+%  dstype %s+% "- Dataset",
         subtitle=paste("AUC =", percent(auc)),
         x="False Positive Rate (1-Specificity)",
         y="True Positive Rate (Sensitivity)")
}

In [37]:
# Assigning varibales for training, validation and testing dataset
mdesc <- "Decision Tree"
dstype_train <- "Training"
dstype_validate <- "Validation"
dstype_test <- "Testing"
train_prob <- predict_prob_class(train)$probability
train_class <- predict_prob_class(train)$class
validate_prob <- predict_prob_class(validate)$probability
validate_class <- predict_prob_class(validate)$class
test_prob <- predict_prob_class(test)$probability
test_class <- predict_prob_class(test)$class


In [38]:
# Evaluate the performance on each data group

m_rp_perf_train <- perf(mdesc, "Training", tr_target, train_prob, train_class)
m_rp_perf_validate <- perf(mdesc, "Validation", va_target, validate_prob, validate_class)
m_rp_perf_test <- perf(mdesc, "Testing", te_target, test_prob, test_class)



Performance Evaluation

Model: Decision Tree
Dataset: Training dataset with 140,000 observations.
 
Overall accuracy: 91.5%
Overall error: 8.48%
 
Error Matrices:

       Predicted
Actual   false true Error
  false 122117 3237   2.6
  true    8628 6018  58.9

       Predicted
Actual  false true Error
  false  87.2  2.3   2.6
  true    6.2  4.3  58.9

Recall: 41%
Precision: 65.2%
F-Score: 0.503
 
Percentage area under the ROC curve AUC: 87.6% 
Performance Evaluation

Model: Decision Tree
Dataset: Validation dataset with 30,000 observations.
 
Overall accuracy: 91.6%
Overall error: 8.37%
 
Error Matrices:

       Predicted
Actual  false true Error
  false 26153  676   2.5
  true   1835 1336  57.9

       Predicted
Actual  false true Error
  false  87.2  2.3   2.5
  true    6.1  4.5  57.9

Recall: 42.5%
Precision: 66.2%
F-Score: 0.517
 
Percentage area under the ROC curve AUC: 87.8% 
Performance Evaluation

Model: Decision Tree
Dataset: Testing dataset with 30,000 observations.
 
Overall 

In [39]:
# Plot the graph for training auc
aucplot(m_rp_perf_train$rates, m_rp_perf_train$auc, mdesc, "Training")
# Plot the graph for validation auc
aucplot(m_rp_perf_validate$rates, m_rp_perf_validate$auc, mdesc, "Validation")
# Plot the graph for testing auc
aucplot(m_rp_perf_test$rates, m_rp_perf_test$auc, mdesc, "Testing")

# Conclusion

* The most important features which helps in predicting if the player would be present in the top 10 is the walking distance and killplace in the game.

* If the person walks a distance of more than ~2600 metres and and has a killplace(Ranking in match of number of enemy players killed) less than 6, better chances of the player making it to the top 10 list.

* Most people prefer to play solo and that too in 'solo first person' category.

## Model Performance Results Summarised

### Training Data Set

* Overall accuracy: 91.5%
* Overall error: 8.48%
* Recall: 41%
* Precision: 65.2%
* F-Score: 0.503
* AUC: 87.6%

#### Validation Data Set

* Overall accuracy: 91.6%
* Overall error: 8.37%
* Recall: 42.5%
* Precision: 66.2%
* F-Score: 0.517
* AUC: 87.8%

#### Testing Data set

* Overall accuracy: 91.4%
* Overall error: 8.56%
* Recall: 41.7%
* Precision: 67.2%
* F-Score: 0.514
* AUC: 87.9%



The model has an overall acurracy of 91.4% and AUC of 87.9%, on testing dataset.

## Improvements and Future Scope

* The precision of the model can be improved to pick the right set of players to be displayed on the online portal using some different modelling techniques like random forest.

* Data can be pruned more to identify cheaters ( eg a person having more than 10 kills without even moving in the game.

* In a game it is not necessary that there would be 100 players everytime, a feature- 'playersJoined' can be introduced and its weightage can be taken into account to see any model improvements.

* The model can be extended to actually predict the probabilty of the person winning the game instead of top 10 position.